# LAB 05 - Data preprocessing

## 01 데이터 재구조화 - Pivot/Melt

- 피벗은 세로형 데이터를 가로형으로 변환하는 것
- 멜트는 가로형 데이터를 세로형으로 변환하는 것


### #01.준비작업


In [1]:
#패키지 참조
from hossam import load_data
from pandas import pivot_table , melt


In [3]:
#샘플 데이터 가져오기
origin = load_data('city_people')
origin

[data] https://data.hossam.kr/data/lab05/city_people.xlsx
[desc] 서울, 인천, 부산에서 2005년, 2010년 2015년에 조사한 가상의 인구수 데이터(인덱스와 메타데이터 없음)
[!] Cannot read metadata


,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,2632035,수도권


### #02.피벗테이블 : (1) 기본 사용 방법 (도시와 연도에 따른 인구수 재배치)
- 리턴되는 피벗 테이블 역시 데이터 프레임 객체

In [ ]:
pivot1=pivot_table(origin,index='도시',columns='연도',values='인구')

print(type(pivot1))  
pivot1

<class 'pandas.core.frame.DataFrame'>


연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,2632035.0,2890451.0


### #02.피벗테이블 : (2) 중복 데이터와 집계 방법 지정하기
- 인덱스와 컬럼에 따른 value 가 두 개 이상인 경우 집계 방법을 지정해야 정확한 결과를 얻을 수 있다

In [5]:
pivot_table(origin,index='지역',columns='연도',values='인구',aggfunc='sum')

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,12263517,12794763


### #02.피벗테이블 : (3) 두 개 이상의 집계 함수 지정
- 집계 함수의 이름을 리스트로 설정한다

In [6]:
pivot_table(origin,index='지역',columns='연도',values='인구',aggfunc=['sum','mean'])

sum                           mean                      
연도      2005      2010      2015       2005       2010       2015
지역                                                               
경상권  3512547   3393191   3448737  3512547.0  3393191.0  3448737.0
수도권  9762546  12263517  12794763  9762546.0  6131758.5  6397381.5

### #02.피벗테이블 : (4) 복수 인덱스 지정


In [7]:
pivot_table(origin,index=['지역','연도'],
            columns='도시',
            values='인구',
            aggfunc=['mean','sum'])

mean                              sum                      
도시               부산         서울         인천         부산         서울         인천
지역  연도                                                                    
경상권 2005  3512547.0        NaN        NaN  3512547.0        NaN        NaN
    2010  3393191.0        NaN        NaN  3393191.0        NaN        NaN
    2015  3448737.0        NaN        NaN  3448737.0        NaN        NaN
수도권 2005        NaN  9762546.0        NaN        NaN  9762546.0        NaN
    2010        NaN  9631482.0  2632035.0        NaN  9631482.0  2632035.0
    2015        NaN  9904312.0  2890451.0        NaN  9904312.0  2890451.0

### #03.Melt
- 데이터 프레임의 컬럼 이름을 변수화 한 형태
- 피벗테이블을 분리한 형태라 볼 수 있음

In [9]:
#샘플 피벗 테이블 생성
pivot_df = pivot_table(origin,index='연도',columns='지역',values='인구',aggfunc='mean')

pivot_df

지역,경상권,수도권
연도,,
2005,3512547.0,9762546.0
2010,3393191.0,6131758.5
2015,3448737.0,6397381.5


In [ ]:
#피벗 테이블 분리 - 1단계 : 데이터 프레임의 인덱스를 일반 컬럼으로 설정한다
pivot_df2 = pivot_df.reset_index()
pivot_df2

지역,연도,경상권,수도권
0,2005,3512547.0,9762546.0
1,2010,3393191.0,6131758.5
2,2015,3448737.0,6397381.5


In [14]:
# 2단계 - 피벗 테이블에 대한 melt 처리
mdf = melt(pivot_df2,id_vars=['연도'],
           value_vars=['경상권','수도권'])

mdf

,연도,지역,value
0,2005,경상권,3512547.0
1,2010,경상권,3393191.0
2,2015,경상권,3448737.0
3,2005,수도권,9762546.0
4,2010,수도권,6131758.5
5,2015,수도권,6397381.5


In [16]:
#피벗 테이블 분리 및 필드 이름 설정 (추후 활용을 고려하면 지정해주는 것 권장)
mdf = melt(pivot_df2,id_vars=['연도'],
           value_vars=['경상권','수도권'],
           var_name='구분',value_name ='인구수')

mdf

,연도,구분,인구수
0,2005,경상권,3512547.0
1,2010,경상권,3393191.0
2,2015,경상권,3448737.0
3,2005,수도권,9762546.0
4,2010,수도권,6131758.5
5,2015,수도권,6397381.5
